<a href="https://colab.research.google.com/github/ali-sdg/chinook_analytics_sqLite3/blob/main/secound_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('chinook.db')


In [ ]:
# ask a keywords from user
keywords_input = input("Enter keywords(EX: Rock, Beatles, Love) — seperate with comma:  ")
# transform str's to list of keywords , del extra spaces(our filter)
keywords = [kw.strip() for kw in keywords_input.split(',') if kw.strip()]


In [ ]:
sql_query = """
SELECT ...
WHERE (t.Name LIKE ? OR ar.Name LIKE ? OR g.Name LIKE ?)
  OR (t.Name LIKE ? OR ar.Name LIKE ? OR g.Name LIKE ?)
  OR (t.Name LIKE ? OR ar.Name LIKE ? OR g.Name LIKE ?)
ORDER BY t.Name;
"""

# if there is no keywords, cand return all records or return a error to user
if not keywords:
    print("you didn't Enter keyword. ")
    conn.close()
    exit()


# a list for hold all the results
all_results = []

for kw in keywords:
    sql = """
    SELECT
      t.TrackId,
      t.Name AS TrackName,
      al.Title AS AlbumTitle,
      ar.Name AS ArtistName,
      g.Name AS GenreName,
      t.UnitPrice
    FROM tracks t
    JOIN albums al ON t.AlbumId = al.AlbumId
    JOIN artists ar ON al.ArtistId = ar.ArtistId
    JOIN genres g ON t.GenreId = g.GenreId
    WHERE t.Name LIKE ? OR ar.Name LIKE ? OR g.Name LIKE ?
    ORDER BY t.Name
    """
    param = (f"%{kw}%", f"%{kw}%", f"%{kw}%")
    df = pd.read_sql_query(sql, conn, params=param)
    if not df.empty:
        df['Keyword'] = kw  #adding a columun for which keyword
        all_results.append(df)

if all_results:
    result_df = pd.concat(all_results).drop_duplicates(subset=['TrackId'])  # Remove duplicates as wishes
    display(result_df.head(50))
else:
    print("no result found. ")

conn.close()



In [ ]:
conn.close()
